<a href="https://colab.research.google.com/github/rajatkb/Intuition-Net_Physics-based-Image-segmentation-outpainting/blob/master/unet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
import keras as K
from keras.models import *
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras



import matplotlib.pyplot as plt
import cv2

from tqdm import tqdm
from PIL import Image


Using TensorFlow backend.


In [0]:
!wget https://www.dropbox.com/s/pxcz2wdz04zxocq/CamVid.zip?dl=1 -O CamVid.zip
!unzip CamVid.zip

--2019-03-27 16:51:09--  https://www.dropbox.com/s/pxcz2wdz04zxocq/CamVid.zip?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.9.1, 2620:100:601f:1::a27d:901
Connecting to www.dropbox.com (www.dropbox.com)|162.125.9.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/pxcz2wdz04zxocq/CamVid.zip [following]
--2019-03-27 16:51:09--  https://www.dropbox.com/s/dl/pxcz2wdz04zxocq/CamVid.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc55c33fe2773d0ea0f096464faa.dl.dropboxusercontent.com/cd/0/get/Ad7SYoFy3gm3rwMVhtgyFh4SCA_ZsgcmyihIaAPMoITISvRFaim6HsRSnFFZFpODQ-5ZIiVnXf2tMba7uoA7HLiuZJrslIbegLbpRCaBvZRYgw/file?dl=1# [following]
--2019-03-27 16:51:09--  https://uc55c33fe2773d0ea0f096464faa.dl.dropboxusercontent.com/cd/0/get/Ad7SYoFy3gm3rwMVhtgyFh4SCA_ZsgcmyihIaAPMoITISvRFaim6HsRSnFFZFpODQ-5ZIiVnXf2tMba7uoA7HLiuZJrslIbegLbpRCaBvZRYgw/file?dl=1
Resolving uc55c33f

In [0]:
def binarylab(labels):
  x = np.zeros([992,320,12])
  for i in range(992):
    for j in range(320):
      x[i,j,labels[i][j]]=1
  return x

def loader(training_path, segmented_path, batch_size, h=320, w=992):
    filenames_t = os.listdir(training_path)
    total_files_t = len(filenames_t)
    
    filenames_s = os.listdir(segmented_path)
    total_files_s = len(filenames_s)
    
    assert(total_files_t == total_files_s)
    
    if str(batch_size).lower() == 'all':
        batch_size = total_files_s
    
    idx = 0
    while(1):
      # Choosing random indexes of images and labels
        batch_idxs = np.random.randint(0, total_files_s, batch_size)
            
        
        inputs = []
        labels = []
        count=0
        for jj in batch_idxs:
          # Reading normalized photo
            img = plt.imread(training_path + filenames_t[jj])
          # Resizing using nearest neighbor method
            img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
            inputs.append(img)
          
          # Reading semantic image
            img = Image.open(segmented_path + filenames_s[jj])
            img = np.array(img)
          # Resizing using nearest neighbor method
            img = cv2.resize(img, (h, w), cv2.INTER_NEAREST)
            labels.append(binarylab(img))
            
        
        
        inputs = np.stack(inputs, axis=2)
        
        yield inputs, np.array(labels)




In [0]:
def get_class_weights(num_classes, c=1.02):
    pipe = loader('/content/train/', '/content/trainannot/', batch_size='all')
    _, labels = next(pipe)
    all_labels = labels.flatten()
    each_class = np.bincount(all_labels, minlength=num_classes)
    prospensity_score = each_class / len(all_labels)
    class_weights = 1 / (np.log(c + prospensity_score))
    return class_weights
  
class_weighting= [0.2595, 0.1826, 4.5640, 0.1417, 0.9051, 0.3826, 9.6446, 1.8418, 6.6823, 6.2478, 3.0, 7.3614]

In [0]:
def get_crop_shape(target, refer):
        # width, the 3rd dimension
        cw = (target.get_shape()[2] - refer.get_shape()[2]).value
        assert (cw >= 0)
        if cw % 2 != 0:
            cw1, cw2 = int(cw/2), int(cw/2) + 1
        else:
            cw1, cw2 = int(cw/2), int(cw/2)
        # height, the 2nd dimension
        ch = (target.get_shape()[1] - refer.get_shape()[1]).value
        assert (ch >= 0)
        if ch % 2 != 0:
            ch1, ch2 = int(ch/2), int(ch/2) + 1
        else:
            ch1, ch2 = int(ch/2), int(ch/2)

        return (ch1, ch2), (cw1, cw2)

def unet(input_size = (320,992,3)):
  concat_axis = 3
  inputs = Input(input_size)
  conv1 = Conv2D(32, (3, 3), padding="same", name="conv1_1", activation="relu", data_format="channels_last")(inputs)
  conv1 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv1)
  pool1 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv1)
  conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool1)
  conv2 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv2)
  pool2 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv2)

  conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool2)
  conv3 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv3)
  pool3 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv3)

  conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool3)
  conv4 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv4)
  pool4 = MaxPooling2D(pool_size=(2, 2), data_format="channels_last")(conv4)

  conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", data_format="channels_last")(pool4)
  conv5 = Conv2D(512, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv5)

  up_conv5 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv5)
  ch, cw = get_crop_shape(conv4, up_conv5)
  crop_conv4 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv4)
  up6   = concatenate([up_conv5, crop_conv4], axis=concat_axis)
  conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(up6)
  conv6 = Conv2D(256, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv6)

  up_conv6 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv6)
  ch, cw = get_crop_shape(conv3, up_conv6)
  crop_conv3 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv3)
  up7   = concatenate([up_conv6, crop_conv3], axis=concat_axis)
  conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(up7)
  conv7 = Conv2D(128, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv7)

  up_conv7 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv7)
  ch, cw = get_crop_shape(conv2, up_conv7)
  crop_conv2 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv2)
  up8   = concatenate([up_conv7, crop_conv2], axis=concat_axis)
  conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(up8)
  conv8 = Conv2D(64, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv8)

  up_conv8 = UpSampling2D(size=(2, 2), data_format="channels_last")(conv8)
  ch, cw = get_crop_shape(conv1, up_conv8)
  crop_conv1 = Cropping2D(cropping=(ch,cw), data_format="channels_last")(conv1)
  up9   = concatenate([up_conv8, crop_conv1], axis=concat_axis)
  conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(up9)
  conv9 = Conv2D(32, (3, 3), padding="same", activation="relu", data_format="channels_last")(conv9)

  #ch, cw = get_crop_shape(inputs, conv9)
  #conv9  = ZeroPadding2D(padding=(ch[0],cw[0]), data_format="channels_last")(conv9)
  #conv10 = Conv2D(1, (1, 1), data_format="channels_last", activation="sigmoid")(conv9)

  conv10 = Conv2D(12, 1, activation = 'softmax')(conv9)
  model = Model(input = inputs, output = conv10)

  model.compile(optimizer = Adam(lr = 1e-4), loss = 'categorical_crossentropy', metrics = ['accuracy'])

  model.summary()

  
  return model


In [0]:
lr = 5e-4
batch_size = 10


print_every = 5
eval_every = 5

In [0]:
train_losses = []
eval_losses = []

bc_train = 367 // batch_size # mini_batch train
bc_eval = 101 // batch_size  # mini_batch validation

# Define pipeline objects
pipe = loader('/content/train/', '/content/trainannot/', batch_size)
eval_pipe = loader('/content/val/', '/content/valannot/', batch_size)

epochs = 100

# Train loop
model=unet(input_size = (320,992,3))
for e in range(1, epochs+1):
    
    print ('-'*15,'Epoch %d' % e, '-'*15)
    
    for _ in tqdm(range(bc_train)):
        X_batch, mask_batch = next(pipe)
        X_batch=np.einsum('klij->ilkj', X_batch)
        mask_batch= np.einsum('klij->kilj',mask_batch)

        print(np.array((mask_batch).shape))
        history = model.fit(X_batch, mask_batch, batch_size=10, nb_epoch=100,verbose=1, class_weight=class_weighting)
        
        model.save_weights('model_weight_ep100.hdf5')
      
               

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("co...)`
  0%|          | 0/36 [00:00<?, ?it/s]

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 320, 992, 3)  0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 320, 992, 32) 896         input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 320, 992, 32) 9248        conv1_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_5 (MaxPooling2D)  (None, 160, 496, 32) 0           conv2d_19[0][0]                  
__________________________________________________________________________________________________
conv2d_20 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:25: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
10/10 [==============================] - 20s 2s/step - loss: 2.5006 - acc: 0.0129
Epoch 2/100
10/10 [==============================] - 3s 281ms/step - loss: 2.4925 - acc: 0.0143
Epoch 3/100
10/10 [==============================] - 3s 271ms/step - loss: 2.4856 - acc: 0.0270
Epoch 4/100
10/10 [==============================] - 3s 271ms/step - loss: 2.4795 - acc: 0.0650
Epoch 5/100
10/10 [==============================] - 3s 271ms/step - loss: 2.4736 - acc: 0.1026
Epoch 6/100
10/10 [==============================] - 3s 269ms/step - loss: 2.4679 - acc: 0.1128
Epoch 7/100
10/10 [==============================] - 3s 270ms/step - loss: 2.4622 - acc: 0.1148
Epoch 8/100
10/10 [==============================] - 3s 270ms/step - loss: 2.4571 - acc: 0.1155
Epoch 9/100
10/10 [==============================] - 3s 270ms/step - loss: 2.4517 - acc: 0.1156
Epoch 10/100
10/10 [==============================] - 3s 271

  3%|▎         | 1/36 [04:56<2:52:54, 296.41s/it]

...

 97%|█████████▋| 35/36 [2:42:30<04:37, 277.91s/it]

[ 10 320 992  12]
Epoch 1/100
10/10 [==============================] - 3s 281ms/step - loss: 0.7613 - acc: 0.8131
Epoch 2/100
10/10 [==============================] - 3s 275ms/step - loss: 0.8495 - acc: 0.7678
Epoch 3/100
10/10 [==============================] - 3s 277ms/step - loss: 0.9905 - acc: 0.7679
Epoch 4/100
10/10 [==============================] - 3s 275ms/step - loss: 0.8998 - acc: 0.7533
Epoch 5/100
10/10 [==============================] - 3s 275ms/step - loss: 0.7396 - acc: 0.7788
Epoch 6/100
10/10 [==============================] - 3s 276ms/step - loss: 0.8769 - acc: 0.7309
Epoch 7/100
10/10 [==============================] - 3s 275ms/step - loss: 0.6465 - acc: 0.8169
Epoch 8/100
10/10 [==============================] - 3s 275ms/step - loss: 0.5879 - acc: 0.8198
Epoch 9/100
10/10 [==============================] - 3s 275ms/step - loss: 0.6973 - acc: 0.7884
Epoch 10/100
10/10 [==============================] - 3s 275ms/step - loss: 0.5976 - acc: 0.8154
Epoch 11/100
10/10 [=

  0%|          | 0/36 [00:00<?, ?it/s]

--------------- Epoch 2 ---------------
[ 10 320 992  12]
Epoch 1/100
10/10 [==============================] - 3s 275ms/step - loss: 0.7283 - acc: 0.8169
Epoch 2/100
10/10 [==============================] - 3s 276ms/step - loss: 0.7202 - acc: 0.8166
Epoch 3/100
10/10 [==============================] - 3s 278ms/step - loss: 0.5974 - acc: 0.8349
Epoch 4/100
10/10 [==============================] - 3s 275ms/step - loss: 0.5373 - acc: 0.8427
Epoch 5/100
10/10 [==============================] - 3s 276ms/step - loss: 0.5267 - acc: 0.8480
Epoch 6/100
10/10 [==============================] - 3s 276ms/step - loss: 0.4821 - acc: 0.8642
Epoch 7/100
10/10 [==============================] - 3s 275ms/step - loss: 0.4761 - acc: 0.8639
Epoch 8/100
10/10 [==============================] - 3s 276ms/step - loss: 0.4378 - acc: 0.8754
Epoch 9/100
10/10 [==============================] - 3s 277ms/step - loss: 0.4583 - acc: 0.8694
Epoch 10/100
10/10 [==============================] - 3s 275ms/step - loss: 0.

KeyboardInterrupt: ignored